In [1]:
# Import packages here

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings #used to remove warning messages in the notebook
warnings.filterwarnings('ignore')

In [2]:
# Import datasets here

train = pd.read_parquet('./data_phase1/train.parquet', engine = 'pyarrow') # train dataset
attributes = pd.read_parquet('./data_phase1/attributes.parquet', engine = 'pyarrow') # attributes dataset

### Data Understanding

We look at the different aspects of the datasets provided to us.

* Majority of the column classes are masked with alphanumeric characters
* The predictor class _is_click_ is available in the train dataset
* The attributes dataset can be joined with the train dataset using a left join to get a holistic picture of the data

In [55]:
train.head()

,query_id,user_id,session_id,product_id,page_type,previous_page_type,device_category,device_platform,user_tier,user_country,context_type,context_value,product_price,week,week_day,is_click
0,92d4dd491a874a2cf92c8d311a44a42b597c64a5ede23d...,e5e4c71b1b9456dafece1338762d4ee3db698cf32c384c...,2740b0d77b4e6fafd75321f7d0794210afa8bd650955e7...,bf056e3841dd3a358c6aacb1f9e74e4c7c4adc62e33b45...,596618814963e496d74434df8b8fe3306892f2e4ce6aaa...,c9f34437ce0e536fefd11a34b9a411b541d2dabfec872a...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,fec032cb05435471f2305006f4a1ba994c9d2f4bcad8ef...,designer_id,e5315dbea15a033bc6974a0bccf5fae4a017648bcd92ab...,0.000263,7,6,0
1,541a93bd95c3f4127a53e6b0d4b41db55ad9cb9e19d34a...,fca847f7eb5a5a21991421354b0f26afb4a517e540541f...,d5feab37634fd140e85b8f98dcb909a8779b4f0417c73a...,c6513ec49c8e04c265c907933799ff76f24c075c6308c4...,06a7f8e972f61aeb0e06335699518079a444e4450ff766...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d179859aac8f7c1f88e1ee29b6655596873318c55127d3...,8c82855f15d05cd74fa59956434df17522fc68e4ce3900...,product_id,61205c20046f2688cb7ed03cad29d5a5dbdc360ff48290...,0.012966,6,5,0
2,263ea1e38126fe0c7bfbff24a33b1a09f4dac4f8cd4bb4...,90ea15d8d96a9d3e7ab463d990e5f4565cac9477498d37...,18ae37a1a05faa7fd54818794a1a8e44073e00a56fa05f...,b4d5e28da10318aa7776b364528dc92f83ba45326018b5...,06a7f8e972f61aeb0e06335699518079a444e4450ff766...,c9f34437ce0e536fefd11a34b9a411b541d2dabfec872a...,bf2241c08d92d32a6782b4041a2c11ca58882ca88454b3...,702e4598004745673c0f6b50387bef9e1d5f503bd8c1c0...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,c94ddcb9053eae77ab9abec807ef2b0bb4efac14404d17...,product_id,3a519c0b692e93feff8810bbbd4654eb297379271a8a04...,0.002734,6,5,0
3,3727580d84ce2fbe42ff8bc6f732331f65ea659864a04c...,8f88d89f2a71e2adf42f885fa6adedd09bf039843b535c...,10667a5a6047aa173d13997cdcd996cbdaf9b0149f9655...,5a36f600d3c01763c28e2dafc53119fba7bcc6a867ab8f...,06a7f8e972f61aeb0e06335699518079a444e4450ff766...,c9f34437ce0e536fefd11a34b9a411b541d2dabfec872a...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d6538f13ace825448d0af4fa5e58d2d08fa2d0850e5e14...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,8c82855f15d05cd74fa59956434df17522fc68e4ce3900...,product_id,c739fd56b5999e40d7391008230454adb1e371d30c6973...,0.000070,5,5,0
4,1fcf5d263785455311cecf2f864eaa2eeca4da488383d9...,6b21688d90dfd9677fa7979dfc6da5b963c0e1e3d68a38...,8b39f00bfc9d45ed6f64dd39a72548936a157f9f03e7f4...,314d20e9e9ec3e97d1867c8bf8c6feb0c23d918021e175...,06a7f8e972f61aeb0e06335699518079a444e4450ff766...,c9f34437ce0e536fefd11a34b9a411b541d2dabfec872a...,bf2241c08d92d32a6782b4041a2c11ca58882ca88454b3...,702e4598004745673c0f6b50387bef9e1d5f503bd8c1c0...,d74a5cebc23c56af60a5768c22d44b52f598629d4011fa...,ea2f413bd8fda0b91a814a68aa520044b204796991a343...,product_id,ebe7ea6e59e7fbd292af5284048b53d356aac5b57d7557...,0.000090,3,4,0


In [56]:
train.columns

Index(['query_id', 'user_id', 'session_id', 'product_id', 'page_type',
       'previous_page_type', 'device_category', 'device_platform', 'user_tier',
       'user_country', 'context_type', 'context_value', 'product_price',
       'week', 'week_day', 'is_click'],
      dtype='object')

In [1]:
# train['product_id'].nunique()

In [58]:
attributes.head()

,product_id,gender,main_colour,second_colour,season,collection,category_id_l1,category_id_l2,category_id_l3,brand_id,season_year,start_online_date,material_values,attribute_values
0,0013f07ccdf212210c110e63f0de46e37669c17a4d855a...,a8c9cca4c116691f1e331a5058e84f05e31696bc4f611c...,7673fc4fdc325f3785a223787d2b32e381e8b4c1c8a765...,4737cd35940c2338e96c18a25aeb6848d46f0da795bce8...,847a067597e39838f1f85b0774f44e68b4d6e64d3ec4dd...,a3791e8d85c005b0d9d60d6d3b7e8edd2f256a5cc928d0...,abf367e49718254e068eee51a565fdcede2d741e4c7e33...,61fe255948ec07c4eb25c70f7144b54beddd00466ad866...,05f872d8b8ec85642ad49786d6e443c0df6e7df4bdcba3...,dd6ea8954a945ef0889f30d57b7fdb8d6aaad397e6c6ff...,c7c4ac6af030e54d02b9e4545e4223e76515c3ce4e498e...,1067.041667,f61ecea9b45f1590e57706b88207449bdd4cb703b917ad...,8b45c5d5e010acf257787f2ce0c505857d94709c436991...
1,002239cd57f19f22e557030dff363dfbd1344d8f7ac829...,4a00d8b84bdb2ec2f219304d3883a46336f9fb38d2f1e6...,0f97dafafa5dc4bb18853ea00776dfcc52302f40411b50...,ca8c396e7422e324d4454a911e0319d07b85a4fb89b006...,9db35d237f5873f0410d3ca18c07430270086eb1e7838d...,a3791e8d85c005b0d9d60d6d3b7e8edd2f256a5cc928d0...,e54f8513b708db3afdbd4950bd3420579a8cddabf4c1b3...,3809cade495cd7dc289e6aee521d380549ebd3456f03bc...,fd021cd2dbaf0d7b6105a1b136cf5a094e025010a2096f...,a6536c6bc250d525ccd3b63a3ec483a33a2010422932a3...,ef7d8d2e02aec8c328dafde95486f7181c37d07be3d167...,215.041667,1675f293342bbb518ba3a5ad39399aa0a13580653d253e...,ca8c396e7422e324d4454a911e0319d07b85a4fb89b006...
2,0028022e1ecbbf92f03a1edb9accb58e7c682e7cd89897...,a8c9cca4c116691f1e331a5058e84f05e31696bc4f611c...,0f97dafafa5dc4bb18853ea00776dfcc52302f40411b50...,ca8c396e7422e324d4454a911e0319d07b85a4fb89b006...,9db35d237f5873f0410d3ca18c07430270086eb1e7838d...,4c8006c7e513057a9138641abc2a9e65f4b014a8045259...,bb660069187af9e9238d10a742def09bf2bed60435b088...,8d4b33036479822fc696f32e1252b16e5105b91b82d564...,ca8c396e7422e324d4454a911e0319d07b85a4fb89b006...,c9c2f76b2ae7911c95e1b9568a614c14bd0eebc750cade...,dc1194ba428d5cd4c49f8a769a6577ac1042162da38bc1...,519.000000,5254281b3c0b606d2c56ae1747cf0b0a868053cc3246d2...,ca8c396e7422e324d4454a911e0319d07b85a4fb89b006...
3,00433de93d9cb6b08584423a6b54306abacef89fbddffe...,a8c9cca4c116691f1e331a5058e84f05e31696bc4f611c...,9a446cf5272a0694254db28c796c058405fc9caeb6352f...,ca8c396e7422e324d4454a911e0319d07b85a4fb89b006...,9db35d237f5873f0410d3ca18c07430270086eb1e7838d...,a3791e8d85c005b0d9d60d6d3b7e8edd2f256a5cc928d0...,abf367e49718254e068eee51a565fdcede2d741e4c7e33...,c71ada9c30cd0210a4bdd2d54172dc7be4f07c9ffdbaa3...,f9226009034d0eecc774be42fbe07e9cdcf9ca5dd24fb5...,2f85e4e89f3d731a3fcc43c1ea068600dc082d9b2656de...,ef7d8d2e02aec8c328dafde95486f7181c37d07be3d167...,202.041667,2a9ca66cf16c629b4d0fb6d78e988a199db1490ef21d78...,ca8c396e7422e324d4454a911e0319d07b85a4fb89b006...
4,009623ea17e53324f8f5a3f45f5b21b9a885ea2765de82...,a8c9cca4c116691f1e331a5058e84f05e31696bc4f611c...,68b6499cff2b4a31b0927effd65c194c69c24954fcb80b...,ca8c396e7422e324d4454a911e0319d07b85a4fb89b006...,9db35d237f5873f0410d3ca18c07430270086eb1e7838d...,b5beb2ed60981746e0b908797b3d3abb8491de15ff5575...,abf367e49718254e068eee51a565fdcede2d741e4c7e33...,c71ada9c30cd0210a4bdd2d54172dc7be4f07c9ffdbaa3...,162f8b725de80863b3ced87304a2922fe3d1bd1f25562f...,668b3eeef29561fa9dd4da956ca3eb2787b2d8df515eed...,ef7d8d2e02aec8c328dafde95486f7181c37d07be3d167...,211.041667,d276da9d2047f312bb486b3b59a646f046d72bc3cf2e19...,ca8c396e7422e324d4454a911e0319d07b85a4fb89b006...


In [59]:
attributes.columns

Index(['product_id', 'gender', 'main_colour', 'second_colour', 'season',
       'collection', 'category_id_l1', 'category_id_l2', 'category_id_l3',
       'brand_id', 'season_year', 'start_online_date', 'material_values',
       'attribute_values'],
      dtype='object')

In [2]:
# attributes['start_online_date'].tail()

In [64]:
all_dfs = [train, attributes]
all_dfs_name = ['train', 'attributes']

for i in range(0,2):
    print("The shape of "+str(all_dfs_name[i])+" is: " +str(all_dfs[i].shape))

The shape of train is: (3507990, 16)
The shape of attributes is: (443150, 14)


Here we factorize the columns for understandability.
Before doing that, we make a copy of the train and attribute datasets (as a checkpoint)

In [3]:
train_copy = train
attributes_copy = attributes

# train_copy['user_country'] = pd.factorize(train_copy.user_country)[0] + 1
# df['Col1'] = pd.factorize(df.Col1)[0] + 1

In [4]:
cols = ['user_id', 'page_type', 'session_id',
       'previous_page_type', 'device_category', 'device_platform', 'user_tier',
       'user_country', 'context_value']

for col in cols:
    train_copy[col] = pd.factorize(train_copy[col])[0] + 1

In [5]:
cols = ['gender', 'main_colour', 'second_colour', 'season',
       'collection', 'category_id_l1', 'category_id_l2', 'category_id_l3',
       'brand_id', 'season_year', 'material_values', 'attribute_values'
       ]

for col in cols:
    attributes_copy[col] = pd.factorize(attributes_copy[col])[0] + 1

In [49]:
train_copy.head()
# attributes_copy.head()

,query_id,user_id,session_id,product_id,page_type,previous_page_type,device_category,device_platform,user_tier,user_country,context_type,context_value,product_price,week,week_day,is_click
0,92d4dd491a874a2cf92c8d311a44a42b597c64a5ede23d...,1,1,bf056e3841dd3a358c6aacb1f9e74e4c7c4adc62e33b45...,1,1,1,1,1,1,designer_id,1,0.000263,7,6,0
1,541a93bd95c3f4127a53e6b0d4b41db55ad9cb9e19d34a...,2,2,c6513ec49c8e04c265c907933799ff76f24c075c6308c4...,2,2,1,1,2,2,product_id,2,0.012966,6,5,0
2,263ea1e38126fe0c7bfbff24a33b1a09f4dac4f8cd4bb4...,3,3,b4d5e28da10318aa7776b364528dc92f83ba45326018b5...,2,1,2,2,1,3,product_id,3,0.002734,6,5,0
3,3727580d84ce2fbe42ff8bc6f732331f65ea659864a04c...,4,4,5a36f600d3c01763c28e2dafc53119fba7bcc6a867ab8f...,2,1,1,1,1,2,product_id,4,0.000070,5,5,0
4,1fcf5d263785455311cecf2f864eaa2eeca4da488383d9...,5,5,314d20e9e9ec3e97d1867c8bf8c6feb0c23d918021e175...,2,1,2,2,1,4,product_id,5,0.000090,3,4,0


In [6]:
leftjoin_df = train_copy.merge(attributes_copy, on = 'product_id', how = 'left')

# left_df.merge(right_df, on='user_id', how='left')

In [68]:
leftjoin_df.head()

,query_id,user_id,session_id,product_id,page_type,previous_page_type,device_category,device_platform,user_tier,user_country,...,season,collection,category_id_l1,category_id_l2,category_id_l3,brand_id,season_year,start_online_date,material_values,attribute_values
0,92d4dd491a874a2cf92c8d311a44a42b597c64a5ede23d...,1,1,bf056e3841dd3a358c6aacb1f9e74e4c7c4adc62e33b45...,1,1,1,1,1,1,...,2,2,10,49,3,314,3,570.041667,47,38
1,541a93bd95c3f4127a53e6b0d4b41db55ad9cb9e19d34a...,2,2,c6513ec49c8e04c265c907933799ff76f24c075c6308c4...,2,2,1,1,2,2,...,1,2,11,170,154,327,2,208.041667,359,2
2,263ea1e38126fe0c7bfbff24a33b1a09f4dac4f8cd4bb4...,3,3,b4d5e28da10318aa7776b364528dc92f83ba45326018b5...,2,1,2,2,1,3,...,2,2,6,11,3,16,3,549.041667,46879,2
3,3727580d84ce2fbe42ff8bc6f732331f65ea659864a04c...,4,4,5a36f600d3c01763c28e2dafc53119fba7bcc6a867ab8f...,2,1,1,1,1,2,...,2,1,4,29,24,1516,3,581.041667,9,42
4,1fcf5d263785455311cecf2f864eaa2eeca4da488383d9...,5,5,314d20e9e9ec3e97d1867c8bf8c6feb0c23d918021e175...,2,1,2,2,1,4,...,2,1,8,14,16,536,3,624.041667,283,4


In [3]:
# Check the % of missing values for all columns, after joining

(leftjoin_df.isna().sum() * 100 / leftjoin_df.shape[0]).sort_values(ascending=False)

NameError: name 'leftjoin_df' is not defined

In [7]:
leftjoin_copy = leftjoin_df

# t[cols].values.tolist()

In [10]:
leftjoin_copy.describe()

,user_id,session_id,page_type,previous_page_type,device_category,device_platform,user_tier,user_country,context_value,product_price,...,season,collection,category_id_l1,category_id_l2,category_id_l3,brand_id,season_year,start_online_date,material_values,attribute_values
count,3.507990e+06,3.507990e+06,3.507990e+06,3.507990e+06,3.507990e+06,3.507990e+06,3.507990e+06,3.507990e+06,3.507990e+06,3.507990e+06,...,3.507990e+06,3.507990e+06,3.507990e+06,3.507990e+06,3.507990e+06,3.507990e+06,3.507990e+06,3.507990e+06,3.507990e+06,3.507990e+06
mean,6.962997e+04,1.274627e+05,1.974367e+00,2.225651e+00,1.570672e+00,1.549487e+00,1.644340e+00,1.903737e+01,5.673028e+04,1.691022e-03,...,1.681874e+00,1.809444e+00,7.191188e+00,4.316450e+01,3.318581e+01,3.375123e+02,2.401336e+00,4.140734e+02,5.979084e+03,3.371523e+01
std,5.722230e+04,8.957139e+04,3.476400e-01,2.031526e+00,5.360738e-01,4.975450e-01,1.333779e+00,2.129745e+01,5.271921e+04,1.245486e-02,...,5.164149e-01,8.658623e-01,6.575287e+00,5.436974e+01,6.522819e+01,4.563977e+02,1.208274e+00,2.999346e+02,1.114775e+04,1.136606e+02
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00
25%,1.998600e+04,4.882200e+04,2.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,4.000000e+00,1.101800e+04,3.295741e-04,...,1.000000e+00,1.000000e+00,4.000000e+00,1.400000e+01,3.000000e+00,4.800000e+01,2.000000e+00,2.090417e+02,6.000000e+01,2.000000e+00
50%,5.487000e+04,1.130060e+05,2.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00,1.000000e+00,1.200000e+01,4.042400e+04,6.202834e-04,...,2.000000e+00,2.000000e+00,6.000000e+00,2.300000e+01,3.000000e+00,1.500000e+02,2.000000e+00,2.800417e+02,4.940000e+02,2.000000e+00
75%,1.099820e+05,1.978760e+05,2.000000e+00,4.000000e+00,2.000000e+00,2.000000e+00,1.000000e+00,2.500000e+01,9.255000e+04,1.166586e-03,...,2.000000e+00,2.000000e+00,8.000000e+00,4.600000e+01,3.000000e+01,4.550000e+02,2.000000e+00,5.320417e+02,5.903000e+03,4.000000e+00
max,2.083930e+05,3.174260e+05,5.000000e+00,2.300000e+01,3.000000e+00,2.000000e+00,6.000000e+00,1.960000e+02,1.895710e+05,1.000000e+00,...,7.000000e+00,9.000000e+00,4.300000e+01,3.940000e+02,5.830000e+02,3.398000e+03,2.000000e+01,3.214042e+03,4.948000e+04,1.469000e+03


In [8]:
# Filling missing values with mode in context_type column

leftjoin_copy['context_type'] = leftjoin_copy['context_type'].fillna(leftjoin_copy['context_type'].mode()[0])

leftjoin_copy.head()

,query_id,user_id,session_id,product_id,page_type,previous_page_type,device_category,device_platform,user_tier,user_country,...,season,collection,category_id_l1,category_id_l2,category_id_l3,brand_id,season_year,start_online_date,material_values,attribute_values
0,92d4dd491a874a2cf92c8d311a44a42b597c64a5ede23d...,1,1,bf056e3841dd3a358c6aacb1f9e74e4c7c4adc62e33b45...,1,1,1,1,1,1,...,2,2,10,49,3,314,3,570.041667,47,38
1,541a93bd95c3f4127a53e6b0d4b41db55ad9cb9e19d34a...,2,2,c6513ec49c8e04c265c907933799ff76f24c075c6308c4...,2,2,1,1,2,2,...,1,2,11,170,154,327,2,208.041667,359,2
2,263ea1e38126fe0c7bfbff24a33b1a09f4dac4f8cd4bb4...,3,3,b4d5e28da10318aa7776b364528dc92f83ba45326018b5...,2,1,2,2,1,3,...,2,2,6,11,3,16,3,549.041667,46879,2
3,3727580d84ce2fbe42ff8bc6f732331f65ea659864a04c...,4,4,5a36f600d3c01763c28e2dafc53119fba7bcc6a867ab8f...,2,1,1,1,1,2,...,2,1,4,29,24,1516,3,581.041667,9,42
4,1fcf5d263785455311cecf2f864eaa2eeca4da488383d9...,5,5,314d20e9e9ec3e97d1867c8bf8c6feb0c23d918021e175...,2,1,2,2,1,4,...,2,1,8,14,16,536,3,624.041667,283,4


In [43]:
leftjoin_copy.columns

Index(['query_id', 'user_id', 'session_id', 'product_id', 'page_type',
       'previous_page_type', 'device_category', 'device_platform', 'user_tier',
       'user_country', 'context_type', 'context_value', 'product_price',
       'week', 'week_day', 'is_click', 'gender', 'main_colour',
       'second_colour', 'season', 'collection', 'category_id_l1',
       'category_id_l2', 'category_id_l3', 'brand_id', 'season_year',
       'start_online_date', 'material_values', 'attribute_values'],
      dtype='object')

In [64]:
# Trial modeling with sample columns

train_trial = leftjoin_copy

In [65]:
train_trial['col_sum'] = train_trial.loc[:,['user_id','session_id', 'page_type', 'previous_page_type', 'device_category', 
                   'device_platform', 'user_tier', 'user_country', 'context_value', 'week', 'week_day',
                   'gender', 'main_colour', 'second_colour', 'season', 'collection', 'category_id_l1',
                   'category_id_l2', 'category_id_l3', 'brand_id', 'season_year', 'material_values', 
                   'attribute_values']].sum(axis=1)

In [66]:
train_trial = train_trial.drop(columns = ['user_id','session_id', 'page_type', 'previous_page_type', 'device_category', 
                   'device_platform', 'user_tier', 'user_country', 'context_value', 'week', 'week_day',
                   'gender', 'main_colour', 'second_colour', 'season', 'collection', 'category_id_l1',
                   'category_id_l2', 'category_id_l3', 'brand_id', 'season_year', 'material_values', 
                   'attribute_values', 'context_type', 'start_online_date'])

In [67]:
train_trial['col_sum'] = np.log(train_trial['col_sum'])

train_trial.head(5)

,query_id,product_id,product_price,is_click,col_sum
0,92d4dd491a874a2cf92c8d311a44a42b597c64a5ede23d...,bf056e3841dd3a358c6aacb1f9e74e4c7c4adc62e33b45...,0.000263,0,6.224558
1,541a93bd95c3f4127a53e6b0d4b41db55ad9cb9e19d34a...,c6513ec49c8e04c265c907933799ff76f24c075c6308c4...,0.012966,0,6.990257
2,263ea1e38126fe0c7bfbff24a33b1a09f4dac4f8cd4bb4...,b4d5e28da10318aa7776b364528dc92f83ba45326018b5...,0.002734,0,10.757201
3,3727580d84ce2fbe42ff8bc6f732331f65ea659864a04c...,5a36f600d3c01763c28e2dafc53119fba7bcc6a867ab8f...,0.000070,0,7.418781
4,1fcf5d263785455311cecf2f864eaa2eeca4da488383d9...,314d20e9e9ec3e97d1867c8bf8c6feb0c23d918021e175...,0.000090,0,6.813445


In [70]:
train_trial_grouped = train_trial.groupby(['query_id', 'product_id']).agg({'col_sum': 'sum'}).reset_index()

In [75]:
train_data_sort = train_trial_grouped.sort_values(['col_sum'])
train_data_sort.head(5)

# train_data_sort['rank'] = train_data_sort['score'].rank(ascending=0, method='first') 

,query_id,product_id,col_sum
2467734,b409e19407f172f1eb3db14303442fe05dc2532ad9b5f8...,8e9e00573faecc9d5dcdccaec2308681e80ac00fbf5132...,5.023881
522820,263ea1e38126fe0c7bfbff24a33b1a09f4dac4f8cd4bb4...,a3af8727270125f4087e87f6b32cdd3f1643dbb6e3f5ef...,5.081404
3459943,fc7f432e797a478a761f6cbb56689b0977c79c52736fe3...,4a7a19acadc3b7a34e45cad12febba2c30c035214de86d...,5.247024
76039,0584998eafc556cda7b6077401b99a4b42a5d51c2c08d5...,67323d723ea3729a27355fb03414eabb95590775861103...,5.379897
1815337,849f0952f4b7152c0b64929039ba2423cbaf3da8f66b2e...,45c78386ba7b7f83b17c3d66e5f36c375df5f7a61db564...,5.429346


In [89]:
train_data_sort['rank'] = train_data_sort.groupby(['query_id', 'product_id'])['col_sum'].rank(method = 'first')

In [90]:
train_data_sort.sort_values(['query_id']).head(100)

,query_id,product_id,col_sum,rank
5,0000437da0efd6e292f12a8a3bf7525de5bd05b5807a74...,aa0bbdfa55326b5c08d3472b1ee1d56fe13a82f63f46c8...,12.587505,1.0
3,0000437da0efd6e292f12a8a3bf7525de5bd05b5807a74...,9e03ca4af958bef1ce5d54e684a40ee7e3a9aa7e5010b6...,12.570549,1.0
0,0000437da0efd6e292f12a8a3bf7525de5bd05b5807a74...,1efd18182268101b62a1ea12a9cafbe05487f3abb92924...,12.570858,1.0
4,0000437da0efd6e292f12a8a3bf7525de5bd05b5807a74...,9e7f6f10b18f35ce924575f599465265e22fa9d75b13c9...,12.570535,1.0
1,0000437da0efd6e292f12a8a3bf7525de5bd05b5807a74...,62349c6eda0dc9fe8bb023213f03ebe93aefa5cbcdfecf...,12.572942,1.0
...,...,...,...,...
93,000277cbba868c6698e0635c57a37bc99bc63b271ebc67...,32f8344ccea67da4718889c2836ac68fb5bbf38de99e55...,12.887269,1.0
100,0002826b012ac2d93a0398ee5f67fff4b3e7f0545db719...,d81476f0b2fd7387f6ed8c84f43728f586f8986d3eddc9...,12.410620,1.0
96,0002826b012ac2d93a0398ee5f67fff4b3e7f0545db719...,4d2149e7e98d677139cb90d866c0d8dfe0bd3d04dc7c47...,12.564617,1.0
101,0002826b012ac2d93a0398ee5f67fff4b3e7f0545db719...,da8c9e6ba67fc4a7f2acb08c003c86a898e3a142da8c22...,12.406439,1.0


In [17]:
# train_trial.groupby('query_id')['is_click'].rank(ascending=False)

0          4.0
1          4.0
2          4.0
3          4.0
4          4.0
          ... 
3507985    4.0
3507986    4.0
3507987    4.0
3507988    4.0
3507989    4.0
Name: is_click, Length: 3507990, dtype: float64

In [94]:
# # Convert the group of all factorized column to a single list column

# cols = ['user_id', 'session_id', 'page_type', 'previous_page_type',
#        'device_category', 'device_platform', 'user_tier', 'user_country',
#        'context_value', 'gender', 'main_colour', 'second_colour', 'season', 
#        'collection', 'category_id_l1', 'category_id_l2', 'category_id_l3',
#        'brand_id', 'season_year', 'material_values', 'attribute_values']

# leftjoin_copy['combined'] = leftjoin_copy[cols].values.tolist()
# leftjoin_copy = leftjoin_copy.drop(cols, axis = 1)

# # df2 = df.drop(cols, axis=1)

In [109]:
# Ranking

leftjoin_copy['rank'] = leftjoin_copy.groupby('query_id')['is_click'].rank(ascending = False)

# df.groupby('Auction_ID')['Bid_Price'].rank(ascending=False)

In [111]:
leftjoin_copy['rank'].unique()

array([4. , 1. , 4.5, 1.5, 5. , 2. , 2.5, 5.5, 3. , 6. ])